# Next-Action and Magnitude Predictor - Example Notebook

This notebook demonstrates the end-to-end usage of the prediction system.


In [ ]:
# Import required libraries
import asyncio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Import predictor modules
from predictor import (
    Config, DbServerProvider, build_features, Predictor, 
    Evaluator, Visualizer, DEFAULT_CONFIG, QUICK_CONFIG, COMPREHENSIVE_CONFIG
)

# Set up plotting
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline


## 1. Configuration Setup

Create a configuration for the prediction system.


In [ ]:
# Create configuration
config = Config(
    symbol="AAPL",
    lookback_days=365,
    horizon_set=["1d", "1w", "1m"],
    timeframe="daily",
    seasonality_years=3,
    models=Config.ModelConfig(
        markov=True,
        gbdt=True,
        logistic_quantile=True,
        hmm=False
    ),
    selection=Config.SelectionConfig(
        validation_window_bars=60,
        blend=True,
        blend_temp=1.0
    ),
    output=Config.OutputConfig(
        ascii=True,
        plots=True,
        export_csv=None
    )
)

print(f"Configuration created for {config.symbol}")
print(f"Horizons: {config.horizon_set}")
print(f"Lookback: {config.lookback_days} days")


## 2. Data Fetching

Connect to the database server and fetch stock data.


In [ ]:
async def fetch_data():
    """Fetch data from the database server."""
    async with DbServerProvider(config.db_host, config.db_port) as db:
        # Check database health
        health = await db.health_check()
        print(f"Database health: {health}")
        
        if not health:
            print("Database server is not healthy. Please ensure db_server.py is running on port 9002.")
            return None
        
        # Fetch daily data
        print(f"Fetching {config.timeframe} data for {config.symbol}...")
        df = await db.get_daily(config.symbol)
        
        if df.empty:
            print(f"No data found for {config.symbol}")
            return None
        
        print(f"Retrieved {len(df)} records")
        print(f"Date range: {df.index.min()} to {df.index.max()}")
        
        return df

# Fetch data
df = await fetch_data()

if df is not None:
    # Display basic info
    print("\nData Summary:")
    print(df.describe())
    
    # Plot price series
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(df.index, df['close'], label='Close Price')
    ax.set_title(f'{config.symbol} Price Series')
    ax.set_ylabel('Price ($)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


## 3. Feature Engineering

Build comprehensive features from the raw stock data.


In [ ]:
if df is not None:
    # Build features
    print("Building features...")
    features_df = build_features(df, config)
    
    print(f"Built features for {len(features_df)} samples")
    print(f"Number of features: {len(features_df.columns)}")
    
    # Display feature summary
    print("\nFeature Summary:")
    print(features_df.describe())
    
    # Plot feature distributions
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Returns distribution
    axes[0, 0].hist(features_df['r1'], bins=50, alpha=0.7)
    axes[0, 0].set_title('Daily Returns Distribution')
    axes[0, 0].set_xlabel('Return')
    axes[0, 0].set_ylabel('Frequency')
    
    # Direction distribution
    direction_counts = features_df['direction'].value_counts()
    axes[0, 1].pie(direction_counts.values, labels=direction_counts.index, autopct='%1.1f%%')
    axes[0, 1].set_title('Direction Distribution')
    
    # Volume distribution
    axes[1, 0].hist(features_df['volume'], bins=50, alpha=0.7)
    axes[1, 0].set_title('Volume Distribution')
    axes[1, 0].set_xlabel('Volume')
    axes[1, 0].set_ylabel('Frequency')
    
    # RSI distribution
    axes[1, 1].hist(features_df['rsi_14'], bins=50, alpha=0.7)
    axes[1, 1].set_title('RSI Distribution')
    axes[1, 1].set_xlabel('RSI')
    axes[1, 1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()


## 4. Model Training

Fit the ensemble of prediction models.


In [ ]:
if df is not None:
    # Create and fit predictor
    print("Fitting predictor models...")
    predictor = Predictor(config)
    predictor.fit(features_df)
    
    # Get model information
    model_info = predictor.get_model_info()
    print(f"\nModels fitted: {model_info['n_models']}")
    print(f"Model names: {model_info['model_names']}")
    print(f"Horizons: {model_info['horizons']}")
    print(f"Blending enabled: {model_info['blend_enabled']}")


## 5. Prediction Generation

Generate predictions for all horizons.


In [ ]:
if df is not None:
    # Generate predictions
    print("Generating predictions...")
    predictions = predictor.predict(features_df, blend=config.selection.blend)
    
    # Display predictions for each horizon
    for horizon in config.horizon_set:
        if horizon in predictions:
            print(f"\n--- HORIZON: {horizon} ---")
            
            pred = predictions[horizon]
            
            # Direction probabilities
            if 'direction_proba' in pred:
                dir_probs = pred['direction_proba']
                print("Direction Probabilities:")
                for direction, probs in dir_probs.items():
                    if len(probs) > 0:
                        print(f"  {direction.upper()}: {probs[-1]:.3f}")
            
            # Expected return
            if 'expected_return' in pred:
                expected_return = pred['expected_return']
                if len(expected_return) > 0:
                    print(f"Expected Return: {expected_return[-1]:.4f} ({expected_return[-1]*100:.2f}%)")
            
            # Quantiles
            if 'quantiles' in pred:
                quantiles = pred['quantiles']
                print("Quantile Predictions:")
                for quantile, values in quantiles.items():
                    if len(values) > 0:
                        print(f"  P{quantile*100:.0f}: {values[-1]:.4f} ({values[-1]*100:.2f}%)")


## 6. Model Evaluation

Evaluate model performance on historical data.


In [ ]:
if df is not None:
    # Create evaluator
    evaluator = Evaluator(config)
    
    # Evaluate all horizons
    print("Evaluating models...")
    evaluation_results = evaluator.evaluate_all_horizons(features_df, predictions)
    
    # Display evaluation results
    for horizon in config.horizon_set:
        if horizon in evaluation_results:
            print(f"\n--- EVALUATION: {horizon} ---")
            
            metrics = evaluation_results[horizon]
            
            # Key metrics
            key_metrics = ['brier_score', 'mae', 'rmse', 'pinball_loss', 'r2']
            for metric in key_metrics:
                if metric in metrics:
                    print(f"{metric.upper()}: {metrics[metric]:.4f}")
    
    # Performance summary
    summary = evaluator.get_performance_summary(evaluation_results)
    print(f"\n--- PERFORMANCE SUMMARY ---")
    print(f"Horizons evaluated: {summary['n_horizons']}")
    
    for metric, stats in summary['metrics'].items():
        print(f"{metric.upper()}:")
        print(f"  Mean: {stats['mean']:.4f}")
        print(f"  Std:  {stats['std']:.4f}")
        print(f"  Min:  {stats['min']:.4f}")
        print(f"  Max:  {stats['max']:.4f}")


## 7. Visualization

Create visualizations of the results.


In [ ]:
if df is not None:
    # Create visualizer
    visualizer = Visualizer(config)
    
    # Feature importance plots
    for horizon in config.horizon_set:
        feature_importance = predictor.get_feature_importance(horizon)
        if feature_importance:
            for model_name, importance in feature_importance.items():
                fig = visualizer.plot_feature_importance(
                    importance, 
                    title=f"Feature Importance - {model_name} - {horizon}"
                )
                plt.show()
    
    # Prediction distribution plots
    for horizon in config.horizon_set:
        if horizon in predictions:
            fig = visualizer.plot_prediction_distribution(
                predictions[horizon],
                title=f"Prediction Distribution - {horizon}"
            )
            plt.show()
    
    # Performance metrics plot
    if evaluation_results:
        fig = visualizer.plot_performance_metrics(
            evaluation_results,
            title=f"Performance Metrics - {config.symbol}"
        )
        plt.show()
    
    # Horizon comparison plot
    if evaluation_results:
        fig = visualizer.plot_horizon_comparison(
            evaluation_results,
            metric='brier_score',
            title=f"Brier Score Comparison - {config.symbol}"
        )
        plt.show()


## 8. Summary

This notebook demonstrated the complete workflow of the Next-Action and Magnitude Predictor:

1. **Configuration**: Set up prediction parameters
2. **Data Fetching**: Connect to database and retrieve stock data
3. **Feature Engineering**: Build comprehensive features from raw data
4. **Model Training**: Fit ensemble of prediction models
5. **Prediction Generation**: Generate predictions for multiple horizons
6. **Model Evaluation**: Assess performance using various metrics
7. **Visualization**: Create plots and charts

The system provides a comprehensive framework for stock prediction with multiple models, horizons, and evaluation metrics.
